In [1]:
from dataloader import *
from model import *

/home/wisla/projects/test-framework/dataloader.py:61: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if config_passed["train_images"] is "flickr8k":
/home/wisla/projects/test-framework/dataloader.py:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr8k_polish":
/home/wisla/projects/test-framework/dataloader.py:74: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr30k":
/home/wisla/projects/test-framework/dataloader.py:80: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr30k_polish":
/home/wisla/projects/test-framework/dataloader.py:87: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "aide":
/home/wisla/projects/test-framework/dataloader.py:94: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "coco17":
/home/wisla/projects/test-

In [2]:
data = DataLoader(config_mixed_coco17_flickr8k)

Photos: train=118287
Loaded: 123287 
Descriptions: train=118287


/home/wisla/projects/test-framework/dataloader.py:61: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if config_passed["train_images"] is "flickr8k":
/home/wisla/projects/test-framework/dataloader.py:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr8k_polish":
/home/wisla/projects/test-framework/dataloader.py:74: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr30k":
/home/wisla/projects/test-framework/dataloader.py:80: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr30k_polish":
/home/wisla/projects/test-framework/dataloader.py:87: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "aide":
/home/wisla/projects/test-framework/dataloader.py:94: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "coco17":
/home/wisla/projects/test-

KeyboardInterrupt: 

In [ ]:
model=ModelImpl(data)

In [ ]:
model.train()

In [ ]:
model.evaluate()